# Module 03: Feature & Target Engineering

## Prerequisites

In [ ]:
# Helper packages
import missingno as msno
import numpy as np
import pandas as pd
from plotnine import ggplot, aes, geom_density, geom_line, geom_point, ggtitle

# Modeling pre-processing with scikit-learn functionality
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA

# Modeling pre-processing with non-scikit-learn packages
from category_encoders.ordinal import OrdinalEncoder
from feature_engine.encoding import RareLabelEncoder

# Modeling
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline

In [ ]:
# Ames housing data
ames = pd.read_csv("../data/ames.csv")

# create train/test split
train, test = train_test_split(ames, train_size=0.7, random_state=123)

# separate features from labels and only use numeric features
X_train = train.drop("Sale_Price", axis=1)
y_train = train[["Sale_Price"]]

# Target engineering

In [ ]:
tt = TransformedTargetRegressor(transformer=PowerTransformer(method='box-cox'))
tt

## Dealing with missingness

### Visualizing missing values

In [ ]:
ames_raw = pd.read_csv("../data/ames_raw.csv")

# count missing values
ames_raw.isnull().sum()

In [ ]:
# can you identify patterns of missing data?
# missingness is represented with white
msno.matrix(ames_raw, labels=True, filter="bottom", sort="ascending", n=50)

In [ ]:
# which features have most missing?
# this chart shows the number of observations so small bars (i.e. Pool QC)
# represent very few observed values (lots of missingness)
msno.bar(ames_raw, labels=True, filter="bottom", sort="ascending", n=50)

### Imputation

#### Estimated statistic

In [ ]:
# median imputation to all features
a = SimpleImputer(strategy='median')

# median imputation to just numeric predictors
b = ColumnTransformer([("num_imp", a, selector(dtype_include="number"))])

# median imputation to 1 or more features
c = ColumnTransformer([("num_imp", a, selector("Gr_Liv_Area"))])

#### K-nearest neighbor

In [ ]:
knn_imp = KNNImputer(n_neighbors=6)

## Feature filtering

In [ ]:
nzv = VarianceThreshold(threshold=0.1)

## Numeric feature engineering

### Skewness

In [ ]:
# Normalizing approach
yj = PowerTransformer(method="yeo-johnson")

# Normalize all numeric features
X_norm = ColumnTransformer([("norm", yj, selector(dtype_include="number"))])

### Standardization

In [ ]:
# Normalizing approach
scaler = StandardScaler()

# standardize all numeric features
std = ColumnTransformer([("norm", scaler, selector(dtype_include="number"))])

## Categorical feature engineering

### One-hot & dummy encoding

In [ ]:
# one-hot encoder
encoder = OneHotEncoder()

# apply to all categorical features
ohe = ColumnTransformer([("one-hot", encoder, selector(dtype_include="object"))])

# dummy encode
encoder = OneHotEncoder(drop='first')

# apply to all categorical features
de = ColumnTransformer([("dummy", encoder, selector(dtype_include="object"))])

### Label encoding

In [ ]:
# create encoder
encoder = LabelEncoder()

# Label encode a single column
lbl = ColumnTransformer([("label", encoder, "MS_SubClass")])

### Ordinal encoding

In [ ]:
# ID all quality features to ordinal encode
cols = list(X_train.filter(regex=("Qual$|QC$|Cond$")).columns)

# specify levels in order
lvs = ["Very_Poor", "Poor", "Fair", "Below_Average", "Average", "Typical", 
       "Above_Average", "Good", "Very_Good", "Excellent", "Very_Excellent"]
val = range(0, len(lvs))

# create a level to integer mapping
lvl_map = dict(zip(lvs, val))
category_mapping = [{'col': col, 'mapping': lvl_map} for col in cols]

# example of first two mappings
category_mapping[0:2]

In [ ]:
# Apply ordinal encoder
cat_encoder = OrdinalEncoder(cols=cols, mapping=category_mapping)

### Lumping

In [ ]:
# create rare label encoder
rare_encoder = RareLabelEncoder(tol=0.01, replace_with="other")

# demonstrate how some neighborhoods are now represented by "other"
rare_encoder.fit_transform(X_train)["Neighborhood"].unique()

## Dimension reduction

In [ ]:
# PCA object - keep 25 components
pca = PCA(n_components=25)

# apply PCA to all numeric features
pca_encoder = ColumnTransformer([("pca", pca, selector(dtype_include="number"))])

## Putting the process together

In [ ]:
# create train/test split
train, test = train_test_split(ames, train_size=0.7, random_state=123)

# separate features from labels and only use numeric features
X_train = train.drop("Sale_Price", axis=1)
y_train = train["Sale_Price"]

# create KNN model object
knn = KNeighborsRegressor()

# define loss function
loss = 'neg_root_mean_squared_error'

# create 10 fold CV object
kfold = KFold(n_splits=10, random_state=123, shuffle=True)

# Create grid of hyperparameter values
hyper_grid = {'knn__n_neighbors': range(2, 26)}

In [ ]:
# 1. Remove near-zero variance features that are categorical  
nzv_encoder = VarianceThreshold(threshold=0.1)

# 2. Ordinal encode our quality-based features 
ord_cols = list(X_train.filter(regex=("Qual$|QC$|Cond$")).columns)
lvs = ["Very_Poor", "Poor", "Fair", "Below_Average", "Average", "Typical", 
       "Above_Average", "Good", "Very_Good", "Excellent", "Very_Excellent"]
val = range(0, len(lvs))
lvl_map = dict(zip(lvs, val))
category_mapping = [{'col': col, 'mapping': lvl_map} for col in ord_cols]
ord_encoder = OrdinalEncoder(cols=ord_cols, mapping=category_mapping)

# 3. Center and scale (i.e., standardize) all numeric features
scaler = StandardScaler()

# 4. Perform dimension reduction by applying PCA to all numeric features
pca = PCA(n_components=30)

# 5. One-hot encode remaining categorical features.
encoder = OneHotEncoder(handle_unknown="ignore")

# combine all steps into a preprocessing pipeline
preprocessor = ColumnTransformer(
  remainder="passthrough",
  transformers=[
  ("nzv_encode", nzv_encoder, selector(dtype_include="number")),
  ("ord_encode", ord_encoder, ord_cols),
  ("std_encode", scaler, selector(dtype_include="number")),
  ("pca_encode", pca, selector(dtype_include="number")),
  ("one-hot", encoder, selector(dtype_include="object")),
  ])

In [ ]:
model_pipeline = Pipeline(steps=[
  ("preprocessor", preprocessor),
  ("knn", knn),
])

In [ ]:
# Tune a knn model using grid search
grid_search = GridSearchCV(model_pipeline, hyper_grid, cv=kfold, scoring=loss)
results = grid_search.fit(X_train, y_train)

# Best model's cross validated RMSE
abs(results.best_score_)

In [ ]:
# Best model's k value
results.best_estimator_.get_params().get('knn__n_neighbors')

In [ ]:
# Plot all RMSE results
all_rmse = pd.DataFrame({'k': range(2, 26), 
                         'RMSE': np.abs(results.cv_results_['mean_test_score'])})

(ggplot(all_rmse, aes(x='k', y='RMSE'))
 + geom_line()
 + geom_point()
 + ggtitle("Cross validated grid search results"))

## Exercises

Using the Ames dataset and the same approach shown in the [last section](#engineering-process-example):

1. Rather than use a 70% stratified training split, try an 80% unstratified training split. How does your cross-validated results compare?

2. Rather than numerically encode the quality and condition features (i.e. `step_integer(matches("Qual|Cond|QC"))`), one-hot encode these features.  What is the difference in the number of features in your training set?  Apply the same cross-validated KNN model to this new feature set. How does the performance change? How does the training time change?

3. Identify three new feature engineering steps that are provided by [recipes](https://recipes.tidymodels.org/), [scikit-learn](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing) or some other open source R/Python package:
   - Why would these feature engineering steps be applicable to the Ames data?
   - Apply these feature engineering steps along with the same cross-validated KNN model. How do your results change?
   
4. Using the [Attrition data set](https://misk-data-science.github.io/misk-homl/docs/01-introduction.nb.html#the_data_sets), assess the characteristics of the target and features.
   - Which target/feature engineering steps should be applied?
   - Create a feature engineering pipeline and apply a KNN grid search. What is the performance of your model?